In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

import dataset_utils
import importlib
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
import dataset_utils
from glob import glob

In [16]:
DATA_DIR = "data/entire_field_raw_3_class"
NUM_POINTS = 100
NUM_CLASSES = 3
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: walking


In [11]:
tf.random.set_seed(7)

In [17]:
# Define the input shape: (sequence_length, num_features)
sequence_length = 100  # Example fixed sequence length
num_features = 5  # 5 input features (x pos, y pos, z pos, energy strength, speed)
num_classes = 2  # Number of classes for classification

# Define the LSTM model
model = models.Sequential()

# LSTM layer
model.add(layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=False))


# Fully connected (Dense) layer
model.add(layers.Dense(32, activation='relu'))

# Output layer with Softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 64)             │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,066 (78.38 KB)

 Trainable params: 20,066 (78.38 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_points, train_labels, epochs=100, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5754 - loss: 0.7045 - val_accuracy: 0.5793 - val_loss: 0.6814
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5754 - loss: 0.6816 - val_accuracy: 0.5793 - val_loss: 0.6775
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5754 - loss: 0.6791 - val_accuracy: 0.5793 - val_loss: 0.6749
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5754 - loss: 0.6766 - val_accuracy: 0.5793 - val_loss: 0.6706
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5794 - loss: 0.6737 - val_accuracy: 0.5793 - val_loss: 0.6640
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5936 - loss: 0.6686 - val_accuracy: 0.5793 - val_loss: 0.6592
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5951 - loss: 0.6654 - val_accuracy: 0.5793 - val_loss: 0.6563
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5859 - loss: 0.6630 - val_accuracy: 0.5793 - v

In [19]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Make predictions on the test set
predictions = model.predict(test_points)

# Convert probabilities to predicted class labels (0, 1, or 2)
predicted_classes = np.argmax(predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(test_labels, predicted_classes)
print("Classification Report:")
print(class_report)

# Evaluate the model's accuracy
accuracy = accuracy_score(test_labels, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Confusion Matrix:
[[57 38]
 [ 5 64]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.60      0.73        95
           1       0.63      0.93      0.75        69

    accuracy                           0.74       164
   macro avg       0.77      0.76      0.74       164
weighted avg       0.80      0.74      0.74       164

Test Accuracy: 0.7378
